In [8]:
!pip install psycopg2

In [9]:
cd '/home/jovyan/lib'

/home/jovyan/lib


In [10]:
pwd

'/home/jovyan/lib'

In [11]:
from db_helper import *

# PART 1

## Data Dictionary for Semantic Search Postgres Database

#### Table Name:  PAGE_INFO
    Column Name: PAGEID
    Column Type: INTEGER PRIMARY KEY
    Column Desc: PAGEID from wikipedia

    Column Name: TITLE
    Column Type: TEXT
    Column Desc: Title of wikipedia webpage

#### Table Name: CATEGORY_INFO
    Column Name: CATEGORYID
    Column Type: INTEGER PRIMARY KEY
    Column Desc: Category Unique identifier.  Postgres auto increment Integer field
    
    Column Name: CATEGORY_NAME
    Column Type: TEXT
    Column Desc: Category Name
        
#### Table Name: PAGE_DATA 
    Column Name: PAGEID 
    Column Type: INTEGER PRIMARY KEY,
    Column Desc: PAGEID from wikipedia
    
    Column Name: TITLE 
    Column Type: TEXT
    Column Desc: wikipedia Page title
    
    Column Name: PAGE_TEXT 
    Column Type: TEXT
    Column Desc: wikipedia page text(cleaned)
        
#### Table Name: CATEGORY_DATA 
    Column Name: CID
    Column Type: INTEGER PRIMARY KEY
    Column Desc: Unique Record identifier.  Postgres auto increment Integer field
    
    Column Name: MAINCATEGORYID 
    Column Type: INTEGER 
    Column Desc: CategoryID of the Main Category
    
    Column Name: PAGEID 
    Column Type: INTEGER
    Column Desc: PAGEID from wikipedia
    
    Column Name: CATEGORY
    Column Type: TEXT
    Column Desc: Category Name
    
    Column Name: SUBCATEGORY 
    Column Type: TEXT
    Column Desc: Sub-Category Name
    
    Column Name: LEVEL 
    Column Type: INTEGER
    Column Desc: sub-level from Main Category

In [ ]:
clear_table = """
BEGIN;
DELETE FROM PAGE_INFO;
COMMIT;
DELETE FROM CATEGORY_INFO;
COMMIT;
DELETE FROM PAGE_DATA;
COMMIT;
DELETE FROM CATEGORY_DATA;
COMMIT;

"""
#only execute if you want to clear the table
#cursor.execute(clear_table)

In [ ]:
connection, cursor = connect_to_db()


def generate_category(category):
    '''
    format a category for insertion in to a wikipedia api call
    '''
    category = re.sub('\s','_',category)
    category = category.lower()
    return category

def generate_query(category):
    '''
    Format an api call for requests
    '''
    query = """
            http://en.wikipedia.org/w/api.php?
            action=query&
            format=json&
            list=categorymembers&
            cmtitle=Category:{}& 
            cmlimit=max
            """.format(generate_category(category))
    query = re.sub('\s','',query)
    return query

def get_page_ids(category, level=0):
    #insert record into the Category Table and get the categoryID
    print('Getting PageIDs from Wikipedia for category: {}.  Please wait'.format(category))
    category = generate_category(category)
    cursor.execute("SELECT CATEGORYID FROM CATEGORY_INFO where category_name = '{}';".format(category))
    cat_id = cursor.fetchall()
    if not cat_id:
        insert_category = """
                BEGIN;
                INSERT INTO CATEGORY_INFO (CATEGORY_NAME) VALUES ('{}');
                COMMIT;
                    """.format(category.lower())
        cursor.execute(insert_category)
    cursor.execute("SELECT CATEGORYID FROM CATEGORY_INFO where category_name = '{}';".format(category))
    cat_id = cursor.fetchall()
    cat_id = cat_id[0]['categoryid']
    get_wikipedia_page_ids(category, cat_id, level)
    print('Finished getting PageID list')
    get_wikipedia_page_text()
    

def get_wikipedia_page_ids(category, cat_id, level = 0):
    response_url = requests.get(generate_query(category))
    response_url.json()
    id_list = response_url.json()['query']['categorymembers']
    for x in id_list:
        pageid = x['pageid']
        title = x['title']
        title = title.replace("'", '"')
        if ('Category:' not in title):
            #check to see if page already exists.  If it does do not insert again
            cursor.execute("""SELECT PAGEID FROM PAGE_INFO WHERE PAGEID = {};""".format(pageid))
            page_exist = cursor.fetchall()
            if not page_exist:
                insert_stmt = """
                       BEGIN;
                       INSERT INTO PAGE_INFO VALUES ({},'{}');
                       COMMIT;
                   """.format(pageid, title)
                cursor.execute(insert_stmt)
            #from here, insert category_id and page_id into category_data table
            
            cursor.execute("""SELECT MAINCATEGORYID FROM CATEGORY_DATA WHERE MAINCATEGORYID = {} AND PAGEID = {};""".format(cat_id, pageid))
            rec_exist = cursor.fetchall()
            if not rec_exist:
                insert_stmt = """
                        BEGIN;
                        INSERT INTO CATEGORY_DATA (MAINCATEGORYID, PAGEID, LEVEL, CATEGORY) VALUES ({}, {}, {}, '{}');
                        COMMIT;
                    """.format(cat_id, pageid,level,category)
                cursor.execute(insert_stmt)
        else:
            # if level > 0 decrease level by 1 and then make recursive call using sub-category as category 
            if level:
                sub_cat = title.replace('Category:', '')
                cursor.execute("""SELECT MAINCATEGORYID, CATEGORY, SUBCATEGORY FROM CATEGORY_DATA
                WHERE MAINCATEGORYID = {} AND CATEGORY = '{}' AND SUBCATEGORY = '{}'""".format(cat_id, category, sub_cat))
                rec_exist = cursor.fetchall()
                if not rec_exist:
                    insert_subcat = """
                        BEGIN;
                        INSERT INTO CATEGORY_DATA (MAINCATEGORYID, CATEGORY, SUBCATEGORY, LEVEL ) VALUES ({}, '{}', '{}',{});
                        COMMIT;
                        """.format(cat_id,  category, sub_cat, level)
                    cursor.execute(insert_subcat)
                    level -=1
                    get_wikipedia_page_ids(sub_cat, cat_id, level)
                    level +=1     
    


def get_wikipedia_page_text():
    print('Start getting wikipedia page text from PageID list.  Please wait')
    select_stmt = """
            SELECT A.PAGEID,A.TITLE FROM PAGE_INFO A LEFT OUTER JOIN PAGE_DATA B ON A.PAGEID = B.PAGEID WHERE B.PAGEID IS NULL;
        """
    cursor.execute(select_stmt)
    result = cursor.fetchall()
    for item in result:
        pageid = item['pageid']
        title = item['title']
        rich_url = 'https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&explaintext=True&pageids=' + str(pageid)
        response_url = requests.get(rich_url)
        r_text = response_url.json()['query']['pages'][str(pageid)]['extract']
        r_text = r_text.replace("'", '"')
        r_text = cleaner(r_text)
        insert_stmt = """
                    BEGIN;
                    INSERT INTO PAGE_DATA VALUES ({},'{}', '{}');
                    COMMIT;
        """.format(pageid, title, r_text)
        cursor.execute(insert_stmt)  
    print('Finished getting page text from wikipedia.  Process is complete')




In [ ]:
get_wikipedia_page_text()

In [ ]:
#Machine Learning level 6
#Business Software level 2
#Maps level 4
get_page_ids('Maps', 4)

# Part 2

In [13]:
def get_data():
    get_some_docs = """
    with tb1 as( SELECT DISTINCT A.PAGEID, A.TITLE, A.PAGE_TEXT, C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
    WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'machine_learning' limit 1100),
    tb2 as 
    (SELECT DISTINCT A.PAGEID, A.TITLE, A.PAGE_TEXT,  C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
    WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'business_software' limit 1100),
    tb3 as
    (SELECT DISTINCT A.PAGEID, A.TITLE, A.PAGE_TEXT,C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
    WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'maps' limit 1100)
    select * from tb1 union all select * from tb2 union all select * from tb3;
    """
    return(query_to_dataframe(get_some_docs))

In [14]:
n_components = 500
algorithm = 'randomized'
random_state = 42
ngram_range=(1,2)
min_df = 1
max_df = .7

#make the pipeline
svd_pipe = Pipeline([
    ('tfidf_vec', TfidfVectorizer( ngram_range, max_df=max_df, min_df=min_df, stop_words = 'english')),
    ('trun_svd', TruncatedSVD(n_components=n_components, algorithm = algorithm, random_state = random_state)),
    ('normalizer', Normalizer(copy=False))
])

#fit and transform model
#after fit and transform the model is trained and ready to find related documents given search term.
df = get_data()
svd_matrix = svd_pipe.fit_transform(df['page_text'])

In [15]:
cd '/home/jovyan/pkl'

/home/jovyan/pkl


In [16]:
#pickle the matrix file so it could be used during prediction
joblib.dump(svd_matrix, 'svd_matrix.pkl')

['svd_matrix.pkl']

In [17]:
#make the pickle of the model
joblib.dump(svd_pipe, 'svd_pipe.pkl') 


['svd_pipe.pkl']

In [18]:
svd_matrix = joblib.load('/home/jovyan/pkl/svd_matrix.pkl')
svd_pipe = joblib.load('/home/jovyan/pkl/svd_pipe.pkl')


In [19]:
#test the code
#make the search query into a query vector
#search_term = ['Data exploration is an approach machine learning is a science']
search_term = ['dot distribution']
#print(search_term)
df = get_data()
query_vector = svd_pipe.transform(search_term)
tmp = pd.DataFrame(np.dot(svd_matrix, query_vector.T))
df['cosine_distance'] = tmp[0]
df.sort_values(['cosine_distance'], ascending=False).head(5)
return_df = pd.DataFrame(df.sort_values(['cosine_distance'], ascending=False).head(5))
return_df.drop(['category_name', 'page_text', 'cosine_distance'], axis=1, inplace=True)
wikipedia_url = 'https://en.wikipedia.org/wiki/'
return_df['title'] = [wikipedia_url + re.sub('\s', '_', i) for i in return_df['title']]
print('Five most relevant wikipedia pages to the search string: {}\n'.format(search_term[0]))
for item in return_df['title']:
    print(item )
print('\n\n')

Five most relevant wikipedia pages to the search string: dot distribution

https://en.wikipedia.org/wiki/Discrete_phase-type_distribution
https://en.wikipedia.org/wiki/Bayesian_hierarchical_modeling
https://en.wikipedia.org/wiki/Dot_distribution_map
https://en.wikipedia.org/wiki/Algorithmic_inference
https://en.wikipedia.org/wiki/Distribution_learning_theory





# Part 3

In [20]:
get_some_docs = """
with tb1 as( SELECT DISTINCT A.PAGEID, A.PAGE_TEXT, C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'machine_learning' limit 1000),
tb2 as 
(SELECT DISTINCT A.PAGEID, A.PAGE_TEXT, C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'business_software' limit 1000),
tb3 as
(SELECT DISTINCT A.PAGEID, A.PAGE_TEXT, C.CATEGORY_NAME FROM PAGE_DATA A, CATEGORY_DATA B, CATEGORY_INFO C
WHERE A.PAGEID = B.PAGEID AND B.MAINCATEGORYID = C.CATEGORYID AND C.CATEGORY_NAME = 'maps' limit 1000)
select * from tb1 union all select * from tb2 union all select * from tb3;
"""

In [21]:
df = query_to_dataframe(get_some_docs)

In [22]:
n_components = 500
algorithm = 'randomized'
random_state = 42
ngram_range=(1,2)
min_df = 1
max_df = .7

tfidf_vec = TfidfVectorizer(ngram_range, max_df = max_df, min_df = min_df, stop_words = 'english')

X = tfidf_vec.fit_transform(df['page_text'])

#make a new DENSE dataframe called text_df from SPARSE matrix X(from tfidf_vectorizer) and put the column headings
#on the dataframe and put the category as index so it can be used easily
text_df = pd.DataFrame(X.toarray(), columns=tfidf_vec.get_feature_names())
text_df.index = df['category_name']

mnb = MultinomialNB()

model = mnb.fit(text_df, text_df.index)

In [23]:
cd '/home/jovyan/pkl'

/home/jovyan/pkl


In [24]:

joblib.dump(tfidf_vec, 'tfidf_vec2.pkl') 
joblib.dump(model, 'prediction_model2.pkl')

['prediction_model2.pkl']

In [25]:
tfidf_vec = joblib.load('/home/jovyan/pkl/tfidf_vec2.pkl')
prediction_model = joblib.load('/home/jovyan/pkl/prediction_model2.pkl')


In [26]:
def generate_page_query(url):
    '''
    Format an api call for requests
    '''
    clean_url = re.sub('\s','_', url.split('/')[-1])
    query = """http://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&explaintext=True&titles={}""".format(clean_url)
    return query

def make_page_prediction(url):
    response_url = requests.get(generate_page_query(url))
    page_id = list(response_url.json()['query']['pages'].keys())[0]
    if int(page_id) == -1:
        print ('Invalid URL given or wikipedia page not found')
    else:
        page_text = response_url.json()['query']['pages'][page_id]['extract']
        X = tfidf_vec.transform([cleaner(page_text)])
        predicted_category = prediction_model.predict(X)[0]   
        proba_dict = {}
        for key, val in enumerate(prediction_model.classes_):
            proba_dict[val] = prediction_model.predict_proba(X)[0][key]
        #print(proba_dict)
        
    return predicted_category, proba_dict[predicted_category].round(3)

In [27]:
url = 'https://en.wikipedia.org/wiki/Dot_distribution_map'
url = 'https://en.wikipedia.org/wiki/R_(programming_language)'
cat, proba = make_page_prediction(url)
print('Page: {} \nPredicted Category: {} \nProbability: {}'.format(url, cat, proba))


Page: https://en.wikipedia.org/wiki/R_(programming_language) 
Predicted Category: machine_learning 
Probability: 0.697
